In [24]:
import os
import openai
from IPython import display

cwd = os.getcwd()

In [4]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
openai.api_key = OPENAI_API_KEY

def run_gpt(messages, this_model="gpt-3.5-turbo"):
    response = openai.ChatCompletion.create(
        model=this_model,
        messages=messages
    )
    return response


def gpt_oneshot(input_prompt, directive="You are a helpful assistant.", verbose=False):
    message_hist = [{"role": "system", "content": directive},  # add directed
                    {"role": "user", "content": input_prompt}]  # init
    response = run_gpt(message_hist)["choices"][0]["message"]["content"]
    if verbose:
        print("chat_gpt: ", response, '\n')
#     message_hist.append({"role": "system", "content": response})
    return response

I am a helpful bot designed to provide information, answer questions, and assist with various topics. Feel free to ask me anything you'd like help with!


In [20]:
# helpers
import base64

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


def get_code(text):
    this_code = []
    for i, code_chunk in enumerate(text.split('```')):  # pieces of code are denoted by ``` so we split
        if i%2 == 1:  # every other chunk is a piece of code in this case
            this_code.append(code_chunk[7:])  # Note: code declarations also have 'python\n' denoting the language, since we dont need this, we omit the first 7 chars
    return this_code

In [41]:
def gpt_image_oneshot(image_link, new_prompt="Please respond with a question for which the answer is the code snippet on this page.",
                      verbose=False):
    new_content = [{"type": "text", "text": "This is the page contains ia block of Verilog code and text relating to it. "+new_prompt},
                   {"type": "image_url", "image_url": {"url": image_link}}]
    message_hist = [{"role": "system", "content": "You are a helpful assistant that gives information on images of code."},
                    {"role": "user", "content": new_content}]  # init
    if verbose:
        print("Asked:", new_content[0]["text"])
#         print("With Image:", new_content[1]["image_url"]["url"])
    response = run_gpt(message_hist, "gpt-4-vision-preview")["choices"][0]["message"]["content"]
    if verbose: print("\nResponded With:", response)
    return response
# TODO account for timout issue
# TODO account for copyright

# test_image1 = cwd+"/testpage.jpg"
# base64_image = encode_image(image_path)
# my_link3 = f"data:image/jpeg;base64,{base64_image}"
# rep = gpt_image_oneshot(my_link3, "Please respond with a question for which the answer is the code snippet on this page.", True)

# print()
# for code_snip in get_code(rep):
#     print(code_snip)
#     print()

Asked: This is the page contains ia block of Verilog code and text relating to it. Please respond with a question for which the answer is the code snippet on this page.

Responded With: What is an example of Verilog code that implements a 4-bit full adder with an immediate assertion for result checking?


In [42]:
# oneshot code
image_to_code = "Respond with only the code present in this image."
image_to_question = "Please respond with a question for which the answer is the code snippet on this page."
code_to_description = "You are a helpful assistant that describes pieces of code."

def info_from_image(image_path, verbose=False):
    """ takes in the link of a local image and gets information about code on it """
    base64_image = encode_image(image_path)
    image_link = f"data:image/jpeg;base64,{base64_image}"
    
    attempts = 0
    raw_code = None  # get gpt to extract code
    while attempts < 3:
        try:
            raw_code = gpt_image_oneshot(image_link, image_to_code, True)
            attempts = 3
        except:
            attempts += 1
    processed_code = '\n'.join(get_code(raw_code))
    
    question = None  # get gpt to ask a question relating to the code
    while question is None:
        try:
            question = gpt_image_oneshot(image_link, image_to_question, True)
        except:
            if verbose: print("Exception occurred on image-to-question")
    
    description = None  # get gpt to describe the code
    desc_prompt = "Write a description for what this piece of Verilog code does:\n" + processed_code
    while description is None:
        try:
            description = gpt_image_oneshot(desc_prompt, code_to_description, True)
        except:
            if verbose: print("Exception occurred on code-to-description")
    
    return [image_path, raw_code, processed_code, question, description]

In [ ]:
# EXAMPLE
tester_im = cwd+"/testpage.jpg"
display.Image(tester_im)
results = info_from_image(tester_im, True)

Asked: This is the page contains ia block of Verilog code and text relating to it. Respond with only the code present in this image.
Exception occurred on image-to-code
Asked: This is the page contains ia block of Verilog code and text relating to it. Respond with only the code present in this image.

Responded With: ```verilog
module adder;
  logic [3:0] a, b, s;
  logic co;
  full_add b0 (.o(s[0]), .a(a[0]), .b(b[0]), .ciN);
  full_add b1 (.o(s[1]), .a(a[1]), .b(b[1]), .ci(o[0]));
  full_add b2 (.o(s[2]), .a(a[2]), .b(b[2]), .ci(o[1]));
  full_add b3 (.o(s[3]), .a(a[3]), .b(b[3]), .ci(o[2]));
  initial begin: testbench
    a = 3'b1; b = 2'd4; co = 0;
    #1 $display("%m a=%b, b=%b, ciN=%b, s=%b, co=%b", a, b, ciN, s, co);
    $time, a = b; ciN = co;
    a = 4'd10; b = 4'd5; co =b  =1'd1;
    #1 $display("%m a=%b, b=%b, ciN=%b, s=%b, co=%b", a, b, ciN, s, co);
    assert (%0==5 && ciN==1)
      $error("%m says no ci gart a=%b, b=%b, ciN=%b, s=%b, co=%b", a, b, ciN, s, co);
    // othe

Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia bl

Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia bl

Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia bl

Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia bl

...

Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia bl

Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia bl

Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia bl

Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia bl

Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia block of Verilog code and text relating to it. You are a helpful assistant that describes pieces of code.
Exception occurred on code-to-description
Asked: This is the page contains ia bl

In [ ]:
print(results[3])
print(results[4])
print()
print(results[2])

In [ ]:
all_images = []
...

In [ ]:
results = 

In [ ]:


# TODO: use this as a baseline for future examples
# TODO: look at all aspects of code
# TODO: assertion-specific dataset and 
# TODO: later, then a more general datas
# raw code datasets --> snippers can be wrong and have bugs - llms are fine-tuned on prompts and responses
# creation of the first "high-quality" prompt-response dataset with sva --> 

# RAW PYTHON, RAW SVA, SUPERVISED SVA (from textbooks)
# Note; should be qualitative analysis with examples --> primarily this is the dataset, the crux is that this is one benefit of using the dataset
# conclude with some basic evaluation of the usefulness of the dataset
# TODO find more textbooks --> 


# TODO get both code and pairs --> CODE, CAPTION, CODE DESCRIPTION, prompt (for which the code presented is a response, aka create a question for which  the code is a response)